
![Title](https://s2.best-wallpaper.net/wallpaper/5120x2880/1908/Squares-geometry-colorful-light-abstract-picture_5120x2880.jpg)

---
## Project in D2424 :  SICOVID-net

[**KTH-DD2424-Github**](https://gits-15.sys.kth.se/rosun/DD2424-Project)






*`Authors:`*


---
> `Roberto Castro Sundin  |` 
[rosun@kth.se](mailto:rosun@kth.se) 

> `Tony Rönnqvist         |` 
[tonyr@kth.se](mailto:tonyr@kth.se)

> `Luis Alejandro Sarmiento González  |`
[lasg@kth.se](mailto:lasg@kth.se)

> `Simon Westberg |`
[swestber@kth.se](mailto:swestber@kth.se)

## 1.3 : Load libraries

In [1]:
# We may not need all the dependencies.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# We import tensorflow, we can do 2.x for 2.0 version
import tensorflow as tf
print(tf.__version__)
from tensorflow.contrib.training import HParams
from tensorflow.python.data import Dataset


# Keras imports, modify or add.
from tensorflow.keras import Model
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization, AveragePooling2D, Input
from tensorflow.keras.layers import DepthwiseConv2D, Lambda, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.compiler.tensorrt import trt_convert as trt

# OPENCV
import cv2

# FIle manipulating libraries.
import pickle
import urllib
import zipfile 
import tarfile
from shutil import copyfile
#import pydicom as dicom

# Math and Data manipulation libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import scipy
import math
import random 

# Imports for plotting.
import itertools
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn

1.15.2-dlenv_tfe


Using TensorFlow backend.


Found GPU at: /device:GPU:0


# 2.0 : Data prepossessing

## 2.1 : Load covid data set


---

In [55]:
# The data is preprocessed by dividing each array with 255 and the normalizing
# the data w.r.t. the training mean and standard deviation

path_pickle = ""

# Training data
X_normal_tr = pickle.load(open(path_pickle + "X_normal_tr.pickle", "rb"))
X_pneumonia_tr = pickle.load(open(path_pickle + "X_pneumonia_tr.pickle", "rb"))
X_covid_tr = pickle.load(open(path_pickle + "X_covid_tr.pickle", "rb"))
# Testing data
X_normal_val = pickle.load(open(path_pickle + "X_normal_val.pickle", "rb"))
X_pneumonia_val = pickle.load(open(path_pickle + "X_pneumonia_val.pickle", "rb"))
X_covid_val = pickle.load(open(path_pickle + "X_covid_val.pickle", "rb"))
# Testing data
X_normal_ts = pickle.load(open(path_pickle + "X_normal_ts.pickle", "rb"))
X_pneumonia_ts = pickle.load(open(path_pickle + "X_pneumonia_ts.pickle", "rb"))
X_covid_ts = pickle.load(open(path_pickle + "X_covid_ts.pickle", "rb"))

## Crop and resize image

In [56]:
# Cropping of images and remove border artifacts

def undo_preprocess(img, X_mean, X_std):
    img = np.multiply(img, X_std) + X_mean
    img *= 255
    return img.astype(dtype=np.int)

def resize_img(img, X_mean, X_std, thickness):
    # Undo the preprocessing
    img = undo_preprocess(img, X_mean, X_std)
    # Resize to smaller dimension
    img = img[thickness:-thickness-1, thickness:-thickness-1]
    # Resize the image back to original dimension
    return cv2.resize(img.astype("float32"), (224, 224), interpolation=cv2.INTER_LINEAR)


thickness = 20
X_mean = pickle.load(open("X_mean.pickle", "rb"))
X_std = pickle.load(open("X_std.pickle", "rb"))


# Resize training set
for i in range(len(X_covid_tr)):
    X_covid_tr[i] = resize_img(X_covid_tr[i], X_mean, X_std, thickness)
    
for i in range(len(X_normal_tr)):
    X_normal_tr[i] = resize_img(X_normal_tr[i], X_mean, X_std, thickness)
    
for i in range(len(X_pneumonia_tr)):
    X_pneumonia_tr[i] = resize_img(X_pneumonia_tr[i], X_mean, X_std, thickness)
    
    
# Resize validation set
for i in range(len(X_covid_val)):
    X_covid_val[i] = resize_img(X_covid_val[i], X_mean, X_std, thickness)
    
for i in range(len(X_normal_val)):
    X_normal_val[i] = resize_img(X_normal_val[i], X_mean, X_std, thickness)
    
for i in range(len(X_pneumonia_val)):
    X_pneumonia_val[i] = resize_img(X_pneumonia_val[i], X_mean, X_std, thickness)
    
    
# Resize testing set
for i in range(len(X_covid_ts)):
    X_covid_ts[i] = resize_img(X_covid_ts[i], X_mean, X_std, thickness)
    
for i in range(len(X_normal_ts)):
    X_normal_ts[i] = resize_img(X_normal_ts[i], X_mean, X_std, thickness)
    
for i in range(len(X_pneumonia_ts)):
    X_pneumonia_ts[i] = resize_img(X_pneumonia_ts[i], X_mean, X_std, thickness)

    
X_tr = np.concatenate((X_normal_tr, X_pneumonia_tr, X_covid_tr), axis=0)
X_mean = np.mean(X_tr, axis=0)
X_std = np.std(X_tr, axis=0, ddof=1)

# Normalize all data w.r.t. training mean and std
X_normal_tr = (X_normal_tr - X_mean) / X_std
X_pneumonia_tr = (X_pneumonia_tr - X_mean) / X_std
X_covid_tr = (X_covid_tr - X_mean) / X_std

X_normal_val = (X_normal_val - X_mean) / X_std
X_pneumonia_val = (X_pneumonia_val - X_mean) / X_std
X_covid_val = (X_covid_val - X_mean) / X_std

X_normal_ts = (X_normal_ts - X_mean) / X_std
X_pneumonia_ts = (X_pneumonia_ts - X_mean) / X_std
X_covid_ts = (X_covid_ts - X_mean) / X_std

path_pickle = "Resized/"

pickle.dump(X_mean, open(path_pickle + "X_mean.pickle", "wb"))
pickle.dump(X_std, open(path_pickle + "X_std.pickle", "wb"))

pickle.dump(X_covid_tr, open(path_pickle + "X_covid_tr.pickle", "wb"))
pickle.dump(X_normal_tr, open(path_pickle + "X_normal_tr.pickle", "wb"))
pickle.dump(X_pneumonia_tr, open(path_pickle + "X_pneumonia_tr.pickle", "wb"))

pickle.dump(X_covid_val, open(path_pickle + "X_covid_val.pickle", "wb"))
pickle.dump(X_normal_val, open(path_pickle + "X_normal_val.pickle", "wb"))
pickle.dump(X_pneumonia_val, open(path_pickle + "X_pneumonia_val.pickle", "wb"))

pickle.dump(X_covid_ts, open(path_pickle + "X_covid_ts.pickle", "wb"))
pickle.dump(X_normal_ts, open(path_pickle + "X_normal_ts.pickle", "wb"))
pickle.dump(X_pneumonia_ts, open(path_pickle + "X_pneumonia_ts.pickle", "wb"))


## 2.2 : Data generator siamese

In [2]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X_normal, X_pneumonia, X_covid, num_channels=1, batch_size=32, samples_per_epoch=5000):
        self.num_channels = num_channels
        self.batch_size = batch_size
        self.samples_per_epoch = samples_per_epoch
        
        self.X_normal = X_normal
        self.X_pneumonia = X_pneumonia
        self.X_covid = X_covid

        self.h = self.X_normal.shape[1]
        self.w = self.X_normal.shape[2]
        
        self.N_normal = self.X_normal.shape[0]
        self.N_pneumonia = self.X_pneumonia.shape[0]
        self.N_covid = self.X_covid.shape[0]

        self.same_combinations = self.get_same_combinations(self.N_covid)
        self.different_combinations = self.get_different_combinations(self.N_covid)

        self.epoch_covid = self.X_covid.reshape(-1, self.h, self.w, num_channels)

        # Extract N_covid amount of normal and pneumonia imgs randomly
        idx_normal = np.random.choice(self.N_normal, size=self.N_covid, replace=False)
        idx_pneumonia = np.random.choice(self.N_pneumonia, size=self.N_covid, replace=False)
        self.epoch_normal = self.X_normal[idx_normal, :, :].reshape(-1, self.h, self.w, self.num_channels)
        self.epoch_pneumonia = self.X_pneumonia[idx_pneumonia, :, :].reshape(-1, self.h, self.w, self.num_channels)

        # Shuffle combinations
        np.random.shuffle(self.same_combinations)
        np.random.shuffle(self.different_combinations)

    def get_same_combinations(self, num):
        lis = []
        for i in range(num):
            for j in range(i+1, num):
                lis.append([i, j])
        return lis

    def get_different_combinations(self, num):
        lis = []
        for i in range(num):
            for j in range(num):
                lis.append([i, j])
        return lis

    def __len__(self):
        """Return number of batches per epoch."""
        return self.samples_per_epoch // self.batch_size
    
    def __getitem__(self, index):
        """Return a batch with corresponding index.
        First batch has index 0, second has index 1, and so on.
        """
        pairs_a = []
        pairs_b = []
        labels = []

        n_same = self.batch_size // 2
        n_different = self.batch_size // 2

        # Generate same-class pairs
        same = self.same_combinations[n_same*index:n_same*(index+1)]

        for i, pair in enumerate(same):
            if i < n_same // 3:
                # Covid
                pairs_a.append(self.epoch_covid[pair[0]])
                pairs_b.append(self.epoch_covid[pair[1]])
            elif i < 2 * (n_same // 3):
                # Normal
                pairs_a.append(self.epoch_normal[pair[0]])
                pairs_b.append(self.epoch_normal[pair[1]])
            elif i < 3 * (n_same // 3):
                # Pneumonia
                pairs_a.append(self.epoch_pneumonia[pair[0]])
                pairs_b.append(self.epoch_pneumonia[pair[1]])
            else:
                # Choose random class for remaining pairs
                k = np.random.randint(3)
                if k == 0:
                    pairs_a.append(self.epoch_covid[pair[0]])
                    pairs_b.append(self.epoch_covid[pair[1]])
                elif k == 1:
                    pairs_a.append(self.epoch_normal[pair[0]])
                    pairs_b.append(self.epoch_normal[pair[1]])
                else:
                    pairs_a.append(self.epoch_pneumonia[pair[0]])
                    pairs_b.append(self.epoch_pneumonia[pair[1]])
            labels.append(0)
        
        # Generate different-class pairs
        different = self.different_combinations[n_different*index:n_different*(index+1)]

        for i, pair in enumerate(different):
            if i < n_different // 3:
                # Covid-normal
                pairs_a.append(self.epoch_covid[pair[0]])
                pairs_b.append(self.epoch_normal[pair[1]])
            elif i < 2 * (n_different // 3):
                # Covid-pneumonia
                pairs_a.append(self.epoch_covid[pair[0]])
                pairs_b.append(self.epoch_pneumonia[pair[1]])
            elif i < 3 * (n_different // 3):
                # Normal-pneumonia
                pairs_a.append(self.epoch_normal[pair[0]])
                pairs_b.append(self.epoch_pneumonia[pair[1]])
            else:
                k = np.random.randint(3)
                if k == 0:
                    # Covid-normal
                    pairs_a.append(self.epoch_covid[pair[0]])
                    pairs_b.append(self.epoch_normal[pair[1]])
                elif k == 1:
                    # Covid-pneumonia
                    pairs_a.append(self.epoch_covid[pair[0]])
                    pairs_b.append(self.epoch_pneumonia[pair[1]])
                else:
                    # Normal-pneumonia
                    pairs_a.append(self.epoch_normal[pair[0]])
                    pairs_b.append(self.epoch_pneumonia[pair[1]])
            labels.append(1)

        pairs_a = np.asarray(pairs_a)
        pairs_b = np.asarray(pairs_b)
        labels = np.asarray(labels)
    
        # Shuffle the data
        permutation = np.random.permutation(len(labels))
        pairs_a = pairs_a[permutation, :, :, :]
        pairs_b = pairs_b[permutation, :, :, :]
        labels = labels[permutation]

        return [pairs_a, pairs_b], labels
  
    def on_epoch_end(self):
        # Extract N_covid amount of normal and pneumonia imgs randomly
        idx_normal = np.random.choice(self.N_normal, size=self.N_covid, replace=False)
        idx_pneumonia = np.random.choice(self.N_pneumonia, size=self.N_covid, replace=False)
        self.epoch_normal = self.X_normal[idx_normal, :, :].reshape(-1, self.h, self.w, self.num_channels)
        self.epoch_pneumonia = self.X_pneumonia[idx_pneumonia, :, :].reshape(-1, self.h, self.w, self.num_channels)

        # Shuffle combinations
        np.random.shuffle(self.same_combinations)
        np.random.shuffle(self.different_combinations)


## 2.3 : Data generator single net

In [2]:
class DataGeneratorSingle(keras.utils.Sequence):
    def __init__(self, X_normal, X_pneumonia, X_covid, num_channels=1, batch_size=32):
        self.num_channels = num_channels
        self.batch_size = batch_size
        
        self.X_normal = X_normal
        self.X_pneumonia = X_pneumonia
        self.X_covid = X_covid

        self.h = self.X_normal.shape[1]
        self.w = self.X_normal.shape[2]
        
        self.N_normal = self.X_normal.shape[0]
        self.N_pneumonia = self.X_pneumonia.shape[0]
        self.N_covid = self.X_covid.shape[0]

        self.epoch_covid = self.X_covid.reshape(-1, self.h, self.w, num_channels)
        np.random.shuffle(self.epoch_covid)

        # Extract N_covid amount of normal and pneumonia imgs randomly
        idx_normal = np.random.choice(self.N_normal, size=self.N_covid, replace=False)
        idx_pneumonia = np.random.choice(self.N_pneumonia, size=self.N_covid, replace=False)
        self.epoch_normal = self.X_normal[idx_normal, :, :].reshape(-1, self.h, self.w, self.num_channels)
        self.epoch_pneumonia = self.X_pneumonia[idx_pneumonia, :, :].reshape(-1, self.h, self.w, self.num_channels)
    
    def __len__(self):
        """Return number of batches per epoch."""
        return (self.N_covid * 3) // self.batch_size
    
    def __getitem__(self, index):
        """Return a batch with corresponding index.
        First batch has index 0, second has index 1, and so on.
        """
        X = []
        labels = []
        n_class = self.batch_size // 3

        idx = np.arange(n_class * index, n_class * (1 + index))

        for i in idx:
            X.append(self.epoch_covid[i])
            X.append(self.epoch_normal[i])
            X.append(self.epoch_pneumonia[i])
            labels.extend([0, 1, 2])
        
        n_imgs = len(labels)

        while n_imgs < self.batch_size:
            k = np.random.randint(2)
            if k == 0:
                idx_normal = np.random.choice(self.N_normal, 1, replace=False)
                X.append(self.X_normal[idx_normal].reshape(self.h, self.w, self.num_channels))
                labels.append(1)
            else:
                idx_pneumonia = np.random.choice(self.N_pneumonia, 1, replace=False)
                X.append(self.X_pneumonia[idx_pneumonia].reshape(self.h, self.w, self.num_channels))
                labels.append(2)
            n_imgs += 1
        
        X = np.asarray(X)
        labels = np.asarray(labels)

        permutation = np.random.permutation(len(labels))
        X = X[permutation, :, :, :]
        labels = labels[permutation]
        return X, labels
    
    def on_epoch_end(self):
        # Extract N_covid amount of normal and pneumonia imgs randomly
        idx_normal = np.random.choice(self.N_normal, size=self.N_covid, replace=False)
        idx_pneumonia = np.random.choice(self.N_pneumonia, size=self.N_covid, replace=False)
        self.epoch_normal = self.X_normal[idx_normal, :, :].reshape(-1, self.h, self.w, self.num_channels)
        self.epoch_pneumonia = self.X_pneumonia[idx_pneumonia, :, :].reshape(-1, self.h, self.w, self.num_channels)
        # Shuffle covid images
        np.random.shuffle(self.epoch_covid)
        

## 2.4 : Generate validation pairs for siamese net

In [ ]:
def get_validation_pairs(trials, X_covid_val, X_normal_val, X_pneumonia_val):
    """ 
    Returns trials amount of validation triplet-pairs for 3-way one shot learning.
    If trials=3, the function returns 
    X_val = [   [[cov_img, cov_img], [cov_img, norm_img], [cov_img, pnem_img]], 
                [[norm_img, cov_img], [norm_img, norm_img], [norm_img, pnem_img]], 
                [[pnem_img, cov_img], [pnem_img, norm_img], [pnem_img, pnem_img]]] 
    Y_val = [   [0, 1, 1], 
                [1, 0, 1], 
                [1, 1, 0]]
    i.e., we get three trials of 3-way one shot learning validation tasks.  
    """

    N_covid_val, h, w = X_covid_val.shape
    N_normal_val = X_normal_val.shape[0]
    N_pneumonia_val = X_pneumonia_val.shape[0]

    if trials % 3 != 0:
        raise ValueError("Trials not a multiple of 3.")

    if trials // 3 >= N_covid_val:
        raise ValueError("Not enough covid images.")

    indices_covid = np.arange(N_covid_val)
    indices_normal = np.arange(N_normal_val)
    indices_pneumonia = np.arange(N_pneumonia_val)

    X_val = []
    Y_val = []

    for i in range(trials // 3):
        # COVID
        # First extract a covid-image to predict the class of
        index = np.random.choice(indices_covid)
        covid_img = X_covid_val[index, :, :].reshape(h, w, 1)
        indices_covid = indices_covid[indices_covid != index]

        # Extract a covid, normal, and pneumonia image to compare against in 3-way learning
        index_covid = np.random.choice(indices_covid)
        img1 = X_covid_val[index_covid, :, :].reshape(h, w, 1)

        index_normal = np.random.choice(indices_normal)
        img2 = X_normal_val[index_normal, :, :].reshape(h, w, 1)

        index_pneumonia = np.random.choice(indices_pneumonia)
        img3 = X_pneumonia_val[index_pneumonia, :, :].reshape(h, w, 1)
        
        # Append the triplet of image-pairs to X_val, and corresponding label to Y_val
        X_val.append([[covid_img, img1], [covid_img, img2], [covid_img, img3]]) 
        Y_val.append([0, 1, 1])
        
        # NORMAL
        # First extract a normal-image to predict the class of
        index = np.random.choice(indices_normal)
        normal_img = X_normal_val[index, :, :].reshape(h, w, 1)
        indices_normal = indices_normal[indices_normal != index]

        # Extract a covid, normal, and pneumonia image to compare against in 3-way learning
        index_covid = np.random.choice(indices_covid)
        img1 = X_covid_val[index_covid, :, :].reshape(h, w, 1)

        index_normal = np.random.choice(indices_normal)
        img2 = X_normal_val[index_normal, :, :].reshape(h, w, 1)

        index_pneumonia = np.random.choice(indices_pneumonia)
        img3 = X_pneumonia_val[index_pneumonia, :, :].reshape(h, w, 1)

        X_val.append([[normal_img, img1], [normal_img, img2], [normal_img, img3]])
        Y_val.append([1, 0, 1])
        
        # PNEUMONIA
        # First extract a pneumonia-image to predict the class of
        index = np.random.choice(indices_pneumonia)
        pneumonia_img = X_pneumonia_val[index, :, :].reshape(h, w, 1)
        indices_pneumonia = indices_pneumonia[indices_pneumonia != index]

        # Extract a covid, normal, and pneumonia image to compare against in 3-way learning
        index_covid = np.random.choice(indices_covid)
        img1 = X_covid_val[index_covid, :, :].reshape(h, w, 1)

        index_normal = np.random.choice(indices_normal)
        img2 = X_normal_val[index_normal, :, :].reshape(h, w, 1)

        index_pneumonia = np.random.choice(indices_pneumonia)
        img3 = X_pneumonia_val[index_pneumonia, :, :].reshape(h, w, 1)

        X_val.append([[pneumonia_img, img1], [pneumonia_img, img2], [pneumonia_img, img3]])
        Y_val.append([1, 1, 0])
    
    return X_val, Y_val

X_val, Y_val = get_validation_pairs(129, X_covid_val, X_normal_val, X_pneumonia_val)
path_pickle = "/content/drive/My Drive/Covid-19_datasets/data/Pickles2/"
#pickle.dump(X_val, open(path_pickle + "X_val.pickle", "wb"), protocol=4)
#pickle.dump(Y_val, open(path_pickle + "Y_val.pickle", "wb"), protocol=4)

# 3.0 :  Network architecture


---

## 3.1 : Network

In [3]:
def PEPX(x, nf1, nf2, nf3, nf4, name):
    # The PEPX-module is described as
    # conv1x1 -> conv1x1 -> DWConv3x3 -> conv1x1 -> conv1x1

    # Project input features to a lower dimension
    # Expand features to a higher dimension that is different than that of the input features 
    # Project features back to a lower dimension, and 
    # Extend channel dimensionality to a higher dimension to produce the ﬁnal features.

    x = Conv2D(filters=nf1, kernel_size=1, strides=1, activation="relu", name=name + "_Project1")(x)
    x = Conv2D(filters=nf2, kernel_size=1, strides=1, activation="relu", name=name + "_Expand")(x)
    x = DepthwiseConv2D(kernel_size=3, strides=1, activation="relu", padding="same", name=name + "_DW")(x)
    x = Conv2D(filters=nf3, kernel_size=1, strides=1, activation="relu", name=name + "_Project2")(x)
    x = Conv2D(filters=nf4, kernel_size=1, strides=1, activation="relu", name=name + "_Extend")(x)
    return x

def covid_net(input_shape, nf, fc_units, single=False):
    """ 
    Replication of the COVID-Net architecture. 
    UL = upper layer in covid-net.
    nf: number of filters in the first PEPX module
    fc_units: number of units in the first fully connected layer
    """
    
    img = Input(shape=input_shape)

    pep_nf = nf

    # Input: 224x224x1, output: 112x112x64
    x = Conv2D(filters=64, kernel_size=7, strides=2, activation="relu", padding="same", name="CV_7x7")(img)

    # UL 1. Input: 112x112x64, output: 56x56xnf
    y1 = Conv2D(filters=pep_nf, kernel_size=1, activation="relu", name="CV_UL1")(x)
    y1 = MaxPool2D(2, name="MP_UL1")(y1)

    # PEPX1.1 - PEPX1.3. 
    # 1.1       Input: 112x112x64, output: 56x56xnf
    # 1.2-1.3.  Input: 56x56xnf, output: 56x56xnf
    pepx1_1 = PEPX(x, 32, 48, 32, pep_nf, "P11")
    pepx1_1 = MaxPool2D(2, name="MP_P11")(pepx1_1)
    
    pepx1_2 = keras.layers.add([pepx1_1, y1])
    pepx1_2 = PEPX(pepx1_2, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P12")

    pepx1_3 = keras.layers.add([pepx1_1, pepx1_2, y1])
    pepx1_3 = PEPX(pepx1_3, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf,  "P13")

    # UL 2. Input: 56x56xnf, output: 28x28x2nf
    y2 = keras.layers.add([pepx1_1, pepx1_2, pepx1_3, y1])
    y2 = Conv2D(filters=2 * pep_nf, kernel_size=1, activation="relu", name="CV_UL2")(y2)
    y2 = MaxPool2D(2, name="MP_UL2")(y2)

    # PEPX 2.1-2.4 
    # 2.1       Input: 56x56xnf, output: 28x28x2nf
    # 2.2-2.4   Input: 28x28x2nf, output: 28x28x2nf
    pepx2_1 = keras.layers.add([pepx1_1, pepx1_2, pepx1_3, y1])
    pepx2_1 = PEPX(pepx2_1, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, 2 * pep_nf, "P21")
    pepx2_1 = MaxPool2D(2, name="MP_P21")(pepx2_1)

    pep_nf *= 2

    pepx2_2 = keras.layers.add([pepx2_1, y2])
    pepx2_2 = PEPX(pepx2_2, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P22")

    pepx2_3 = keras.layers.add([pepx2_1, pepx2_2, y2])
    pepx2_3 = PEPX(pepx2_3, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P23")

    pepx2_4 = keras.layers.add([pepx2_1, pepx2_2, pepx2_3, y2])
    pepx2_4 = PEPX(pepx2_4, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P24")

    # UL 3. Input: 28x28x2nf, output: 14x14x4nf
    y3 = keras.layers.add([pepx2_1, pepx2_2, pepx2_3, pepx2_4, y2])
    y3 = Conv2D(filters=2 * pep_nf, kernel_size=1, activation="relu", name="CV_UL3")(y3)
    y3 = MaxPool2D(2, name="MP_UL3")(y3)

    # PEPX 3.1-3.6. 
    # 3.1       Input: 28x28x2nf, output: 14x14x4nf
    # 3.2-3.6   Input: 14x14x4nf, output: 14x14x4nf
    pepx3_1 = keras.layers.add([pepx2_1, pepx2_2, pepx2_3, pepx2_4, y2])
    pepx3_1 = PEPX(pepx3_1, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, 2 * pep_nf, "P31")
    pepx3_1 = MaxPool2D(2, name="MP_P31")(pepx3_1)

    pep_nf *= 2

    pepx3_2 = keras.layers.add([pepx3_1, y3])
    pepx3_2 = PEPX(pepx3_2, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P32")

    pepx3_3 = keras.layers.add([pepx3_1, pepx3_2, y3])
    pepx3_3 = PEPX(pepx3_3, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P33")

    pepx3_4 = keras.layers.add([pepx3_1, pepx3_2, pepx3_3, y3])
    pepx3_4 = PEPX(pepx3_4, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P34")

    pepx3_5 = keras.layers.add([pepx3_1, pepx3_2, pepx3_3, pepx3_4, y3])
    pepx3_5 = PEPX(pepx3_5, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P35")

    pepx3_6 = keras.layers.add([pepx3_1, pepx3_2, pepx3_3, pepx3_4, pepx3_5, y3])
    pepx3_6 = PEPX(pepx3_6, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P36")

    # UL 4. Input: 14x14x4nf, output: 7x7x8nf
    y4 = keras.layers.add([pepx3_1, pepx3_2, pepx3_3, pepx3_4, pepx3_5, pepx3_6, y3])
    y4 = Conv2D(filters=2 * pep_nf, kernel_size=1, activation="relu", name="CV_UL4")(y4)
    y4 = MaxPool2D(2, name="MP_UL4")(y4)

    # PEPX 4.1-4.3
    # 4.1       Input: 14x14x4nf, output: 7x7x8nf
    # 4.2-4.3   Input: 7x7x8nf, output: 7x7x8nf
    pepx4_1 = keras.layers.add([pepx3_1, pepx3_2, pepx3_3, pepx3_4, pepx3_5, pepx3_6, y3])
    pepx4_1 = PEPX(pepx4_1, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, 2 * pep_nf, "P41")
    pepx4_1 = MaxPool2D(2, name="MP_P41")(pepx4_1)

    pep_nf *= 2

    pepx4_2 = keras.layers.add([pepx4_1, y4])
    pepx4_2 = PEPX(pepx4_2, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P42")

    pepx4_3 = keras.layers.add([pepx4_1, pepx4_2, y4])
    pepx4_3 = PEPX(pepx4_3, pep_nf // 2, 3 * pep_nf // 4, pep_nf // 2, pep_nf, "P43")

    # Flatten and two fully connected layers, possibly dropout
    z = keras.layers.add([pepx4_1, pepx4_2, pepx4_3, y4], name="OUTPUTX")
    
    flat = Flatten(name="Flat")(z)   # 100352x1
    flat = keras.layers.Dropout(rate=0.50)(flat)
    
    fc1 = Dense(units=fc_units, activation="relu", name="FC1")(flat)
    fc1 = keras.layers.Dropout(rate=0.20)(fc1)
    
    fc2 = Dense(units=256, activation="relu", name="FC2")(fc1)
    fc2 = keras.layers.Dropout(rate=0.20)(fc2)
    
    if single:
        output = Dense(units=3, name="Output_before_SM")(fc2)
        output = Activation("softmax", name="Output_after_SM")(output)
        return Model(img, output)
        

    return Model(img, fc2)

def siamese_net(nf, fc_units):
    input_shape = (224, 224, 1)
    
    covid_model = covid_net(input_shape, nf, fc_units)
    
    img1 = Input(shape=input_shape, name="Img1")  # First image
    img2 = Input(shape=input_shape, name="Img2")  # Second image
    
    feature_vec1 = covid_model(img1)
    feature_vec2 = covid_model(img2)

    l1 = Lambda(lambda tensors : keras.backend.abs(tensors[0] - tensors[1]), name="Lambda")([feature_vec1, feature_vec2])
    similarity = Dense(units=1, activation="sigmoid", name= "Similarity")(l1)
    
    model = Model(inputs=[img1, img2], outputs=similarity) 
    return model


## 3.2 : 3-way validation callback

In [7]:
class Validation(keras.callbacks.Callback):
    """A custom validation callback for 3-way one-shot validation used in the 
    training of a siamese network."""
    
    def __init__(self, validation_data, tr_batches_per_epoch):
        super(keras.callbacks.Callback, self).__init__()
        self.validation_data = validation_data
        self.X_val = self.validation_data[0]
        self.Y_val = self.validation_data[1]
        self.trials = len(self.Y_val)
        self.val_accuracy = []
        self.tr_batches_per_epoch = tr_batches_per_epoch
        
        self.max_val_acc = 0

        self.confusion_matrices = []
        self.confusion_matrices_perc = []

    def on_epoch_end(self, epoch, logs=None):
        self.validation()
        accuracy = self.val_accuracy[-1]
        print("\n3-way validation accuracy {} trials - epoch: {} - accuracy: {}".format(self.trials, epoch, accuracy))
        print(self.confusion_matrices_perc[-1])
        
        # Save model if accuracy is larger than previous max validation accuracy
        #if accuracy > self.max_val_acc:
        #    self.max_val_acc = accuracy
        #    self.model.save("Models/siameseNet")

    def validation(self):
        correct = 0
        true_labels = []
        predicted_labels = []
        for i, triplet in enumerate(self.X_val):
          
            pair1 = triplet[0]
            pair2 = triplet[1]
            pair3 = triplet[2]

            sim1 = self.model.predict(x=[[pair1[0]], [pair1[1]]])
            sim2 = self.model.predict(x=[[pair2[0]], [pair2[1]]])
            sim3 = self.model.predict(x=[[pair3[0]], [pair3[1]]])
            
            predict_triplet = [sim1, sim2, sim3]
            Y_triplet = self.Y_val[i]
            
            prediction = np.argmin(predict_triplet)
            truth = np.argmin(Y_triplet)
            
            true_labels.append(truth)
            predicted_labels.append(prediction)

            if prediction == truth:
                correct += 1
        
        accuracy = correct / self.trials
        conf = confusion_matrix(true_labels, predicted_labels)
        self.val_accuracy.append(np.round(accuracy, 3))
        self.confusion_matrices.append(conf)
        self.confusion_matrices_perc.append(np.round(conf / np.sum(conf, axis=1), 2))


# 4.0 : Training and validation

## 4.1 : Hyperparameters

In [4]:
# Set hyperparameters before training
hp = HParams(
    # Network
    nf=128,
    fc_units=1024,
    # Training
    learning_rate=1e-4,
    batch_size_tr=16,
    batch_size_val=16,
    samples_per_epoch_tr=12000,
    samples_per_epoch_val=1000,
    n_epochs=100,
    workers=1,
    # Directories
    path_pickle="Resized/",
    path_figs="Figures/",
    checkpoint_dir='/tmp/checkpoints/ '
)

## 4.2 : Load training and validation data

In [7]:
# Load training and validation data to feed to data generator
X_normal_tr = pickle.load(open(hp.path_pickle + "X_normal_tr.pickle", "rb"))
X_pneumonia_tr = pickle.load(open(hp.path_pickle + "X_pneumonia_tr.pickle", "rb"))
X_covid_tr = pickle.load(open(hp.path_pickle + "X_covid_tr.pickle", "rb"))

X_normal_val = pickle.load(open(hp.path_pickle + "X_normal_val.pickle", "rb"))
X_pneumonia_val = pickle.load(open(hp.path_pickle + "X_pneumonia_val.pickle", "rb"))
X_covid_val = pickle.load(open(hp.path_pickle + "X_covid_val.pickle", "rb"))

# Validation for single COVID-net
X_val_single = pickle.load(open(hp.path_pickle + "X_val_single.pickle", "rb"))
Y_val_single = pickle.load(open(hp.path_pickle + "Y_val_single.pickle", "rb"))

# Load validation data to use in 3-way validation callback
X_val = pickle.load(open(hp.path_pickle + "X_val.pickle", "rb"))
Y_val = pickle.load(open(hp.path_pickle + "Y_val.pickle", "rb"))

## 4.3 : Train siamese model

In [ ]:
# SIAMESE NET


learning_rates = [1e-4]
batch_sizes = [32]

for bz in batch_sizes:
    for lr in learning_rates:
        print("bz:", bz, "lr:", lr)
        print()

        model = siamese_net(nf=hp.nf, fc_units=hp.fc_units)
        adam = keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=adam, loss="binary_crossentropy", metrics=["accuracy"])

        training_generator = DataGenerator(X_normal_tr, X_pneumonia_tr, X_covid_tr, num_channels=1, 
                                           batch_size=bz, samples_per_epoch=hp.samples_per_epoch_tr)

        val_callback = Validation(validation_data=(X_val, Y_val), tr_batches_per_epoch=training_generator.__len__())

        history = model.fit_generator(generator=training_generator, 
                                      epochs=hp.n_epochs, callbacks=[val_callback],
                                      use_multiprocessing=False, workers=hp.workers)
        d = history.history
        d_3w = {"val_acc_3w": val_callback.val_accuracy, "conf": val_callback.confusion_matrices_perc}
        d_combined = {**d, **d_3w}

        pickle.dump(d_combined, open(hp.path_pickle + "histories_siamese_final.pickle", "wb"))
        
#pickle.dump(history.history, open(hp.path_pickle + "history.pickle", "wb"))
#pickle.dump(val_callback.val_accuracy, open(hp.path_pickle + "val_acc.pickle", "wb"))
#pickle.dump(val_callback.confusion_matrices_perc, open(hp.path_pickle + "conf.pickle", "wb"))

## 4.4 : Train single model

In [ ]:
# SINGLE NET

model = covid_net_single(nf=hp.nf, fc_units=hp.fc_units, single=True)
adam = keras.optimizers.Adam(learning_rate=hp.learning_rate)
model.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

save_callback = keras.callbacks.ModelCheckpoint("Models/singleNet_resized", monitor='val_acc', save_best_only=True, save_weights_only=False)
#tb_callback = keras.callbacks.TensorBoard(log_dir="Logs")

training_generator = DataGeneratorSingle(X_normal_tr, X_pneumonia_tr, X_covid_tr, num_channels=1,
                                 batch_size=hp.batch_size_tr)

history = model.fit_generator(generator=training_generator, validation_data=(X_val_single, Y_val_single),
                                  epochs=hp.n_epochs, callbacks=[save_callback], use_multiprocessing=False, workers=hp.workers)


#pickle.dump(history.history, open(hp.path_pickle + "history.pickle", "wb"))
#pickle.dump(val_callback.val_accuracy, open(hp.path_pickle + "val_acc.pickle", "wb"))
#pickle.dump(val_callback.confusion_matrices_perc, open(hp.path_pickle + "conf.pickle", "wb"))

## 4.5 : Confusion matrix

In [ ]:
#history = pickle.load(open(hp.path_pickle + "history.pickle", "rb"))
#val_acc = pickle.load(open(hp.path_pickle + "val_acc.pickle", "rb"))
val_acc_3w = val_callback.val_accuracy

best = int(np.argmax(val_acc_3w))
best_val_acc = val_acc_3w[best]
best_conf = val_callback.confusion_matrices_perc[best]

print("Highest val acc:", best_val_acc)

df_cm = pd.DataFrame(best_conf, index=["covid", "normal", "pneumonia"],  columns=["covid", "normal", "pneumonia"])
seaborn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.savefig(hp.path_figs + "fig3_conf.pdf")

## 4.6 : Plotting

In [5]:
import pickle
import numpy as np

histories = pickle.load(open("histories_siamese_final.pickle", "rb"))
histories_single = pickle.load(open("histories_128_single.pickle", "rb"))

0.736
28


In [ ]:
# Siamese
histories = pickle.load(open(hp.path_pickle + "history.pickle", "rb"))

epochs = np.arange(1, hp.n_epochs + 1)
loss = histories["loss"]
acc = histories["acc"]
val_acc_3w = histories["val_acc"]

plt.subplot(2, 1, 1)
plt.plot(epochs, loss, label="training loss")
plt.ylabel("loss")

plt.tick_params(
    axis='x',
    which='both', 
    bottom=False, 
    top=False,
    labelbottom=False)

plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, label="training accuracy")
plt.plot(epochs, val_acc_3w, label="3-way validation accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend(fontsize="x-small", loc=(0.56, 0.55))

plt.tight_layout()
plt.savefig(hp.path_figs + "siameseFinalRun.pdf")
plt.show()


In [ ]:
# Single net
histories = pickle.load(open(hp.path_pickle + "history.pickle", "rb"))

epochs = np.arange(1,  101)
loss = histories["loss"]
acc = histories["acc"]
val_loss = histories["val_loss"]
val_acc = histories["val_acc"]

plt.subplot(2, 1, 1)
plt.plot(epochs, loss, label="training loss")
plt.plot(epochs, val_loss, label="validation loss")
plt.ylabel("loss")

plt.tick_params(
    axis='x',
    which='both', 
    bottom=False, 
    top=False,
    labelbottom=False)

plt.legend(fontsize="x-small", loc="upper left")

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, label="training accuracy")
plt.plot(epochs, val_acc, label="validation accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.yticks([0.6, 0.7, 0.8, 0.9])
plt.legend(fontsize="x-small", loc="lower right")

plt.tight_layout()
plt.savefig(hp.path_figs + "singleFinalRun_cropped.pdf")
plt.show()


# 5.0 Testing

## 5.1 Load data

In [11]:
# Training data is used to create class templates for testing the siamese network
X_normal_tr = pickle.load(open(hp.path_pickle + "X_normal_tr.pickle", "rb"))
X_pneumonia_tr = pickle.load(open(hp.path_pickle + "X_pneumonia_tr.pickle", "rb"))
X_covid_tr = pickle.load(open(hp.path_pickle + "X_covid_tr.pickle", "rb"))

# Test data
X_normal_ts = pickle.load(open(hp.path_pickle + "X_normal_ts.pickle", "rb"))
X_pneumonia_ts = pickle.load(open(hp.path_pickle + "X_pneumonia_ts.pickle", "rb"))
X_covid_ts = pickle.load(open(hp.path_pickle + "X_covid_ts.pickle", "rb"))

N_covid_tr = X_covid_tr.shape[0]
N_normal_tr = X_normal_tr.shape[0]
N_pneumonia_tr = X_pneumonia_tr.shape[0]

N_covid_val = X_covid_val.shape[0]
N_normal_val = X_normal_val.shape[0]
N_pneumonia_val = X_pneumonia_val.shape[0]

N_covid_ts = X_covid_ts.shape[0]
N_normal_ts = X_normal_ts.shape[0]
N_pneumonia_ts = X_pneumonia_ts.shape[0]

## 5.2 Load models

In [138]:
# Pretrained models, both single covid-net and a siamese net
model_siamese = keras.models.load_model("Models/siameseNet")
model_single_resized = keras.models.load_model("Models/singleNet_resized")
model_single = keras.models.load_model("Models/singleNet")

## 5.3 Test single model

In [139]:
# Concatenate test data
X_ts = np.concatenate((X_covid_ts_R, X_normal_ts_R, X_pneumonia_ts_R)).reshape(-1, 224, 224, 1)

Y_ts = [0 for i in range(N_covid_ts)]
Y_ts.extend([1 for i in range(N_normal_ts)])
Y_ts.extend([2 for i in range(N_pneumonia_ts)])

Y_ts = np.asarray(Y_ts)

predictions = model_single_resized.predict(X_ts)
Y_predicted = []

for pred in predictions:
        Y_predicted.append(np.argmax(pred))



In [ ]:
# Confusion matrix total numbers, single net
conf = confusion_matrix(Y_ts, Y_predicted) 

# Total accuracy
print((conf[0, 0] + conf[1, 1] + conf[2, 2]) / np.sum(conf))

seaborn.set(font_scale=1.1) 
df_cm = pd.DataFrame(conf, index=["C", "N", "P"],  columns=["C", "N", "P"])
ax = seaborn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt="d")
ax.set_xlabel("Predicted label", fontsize=14)
ax.set_ylabel("True label", fontsize=14)
plt.yticks(rotation=0)
plt.savefig(hp.path_figs + "confSingleNet_numbers_cropped.pdf")

In [ ]:
# Confusion matrix percentage for single net. Total percentage 81% 
conf = np.round(conf / np.sum(conf, axis=1).reshape(-1, 1), 2)

seaborn.set(font_scale=1.1) 
df_cm = pd.DataFrame(conf, index=["C", "N", "P"],  columns=["C", "N", "P"])
ax = seaborn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
ax.set_xlabel("Predicted label", fontsize=14)
ax.set_ylabel("True label", fontsize=14)
plt.yticks(rotation=0)
plt.savefig(hp.path_figs + "confSingleNet_78p_cropped.pdf")

## 5.5 Generate class templates for siamese model

In [108]:
# Create a model that returns the feature vector calculated by the covid-net
input1 = model_siamese.layers[2].get_input_at(0)
output1 = model_siamese.layers[2].get_output_at(0)
model_feat_vec = Model(inputs=input1, outputs=output1)

# Create tempalates as the average over all feature vectors calculated from the training data
covid_template = np.zeros((1, 256))
normal_template = np.zeros((1, 256))
pneumonia_template = np.zeros((1, 256))

for i in range(N_covid_tr):
    covid_img = X_covid_tr[i].reshape(224, 224, 1)
    covid_vec = model_feat_vec.predict([[covid_img]])
    covid_template += covid_vec

for i in range(N_normal_tr):
    normal_img = X_normal_tr[i].reshape(224, 224, 1)
    normal_vec = model_feat_vec.predict([[normal_img]])
    normal_template += normal_vec

for i in range(N_pneumonia_tr):
    pneumonia_img = X_pneumonia_tr[i].reshape(224, 224, 1)
    pneumonia_vec = model_feat_vec.predict([[pneumonia_img]])
    pneumonia_template += pneumonia_vec

covid_template /= N_covid_tr
normal_template /= N_normal_tr
pneumonia_template /= N_pneumonia_tr

#pickle.dump(covid_template, open(hp.path_pickle + "covid_template.pickle", "wb"))
#pickle.dump(normal_template, open(hp.path_pickle + "normal_template.pickle", "wb"))
#pickle.dump(pneumonia_template, open(hp.path_pickle + "pneumonia_template.pickle", "wb"))

## 5.6 Test siamese network

In [ ]:
def test_siamese(model, X_covid_ts, X_normal_ts, X_pneumonia_ts, covid_template, normal_template, pneumonia_template):
    """Functions to test the accuracy of the siamese network on the test data. For each test image, a similarity score
    is calculated between the image and all three class templates. The image is predicted to have the same class as
    the class that produced the smallest similarity measure (since same class images should have output 0 and different classes
    should have output 1).
    """
    
    N_covid = X_covid_ts.shape[0]
    N_normal = X_normal_ts.shape[0]
    N_pneumonia = X_pneumonia_ts.shape[0]
    
    predictions = []
    correct = []
    
    for i in range(N_covid):
        img_covid = X_covid_ts[i].reshape(224, 224, 1)
        
        covid_sim = model.predict([[img_covid], covid_template])
        normal_sim = model.predict([[img_covid], normal_template])
        pneumonia_sim = model.predict([[img_covid], pneumonia_template])
        
        sim = [covid_sim, normal_sim, pneumonia_sim]
        
        correct.append(0)
        predictions.append(np.argmin(sim))
    
    for i in range(N_normal):
        img_normal = X_normal_ts[i].reshape(224, 224, 1)
        
        covid_sim = model.predict([[img_normal], covid_template])
        normal_sim = model.predict([[img_normal], normal_template])
        pneumonia_sim = model.predict([[img_normal], pneumonia_template])
        
        sim = [covid_sim, normal_sim, pneumonia_sim]
        
        correct.append(1)
        predictions.append(np.argmin(sim))
    
    for i in range(N_pneumonia):
        img_pneumonia = X_pneumonia_ts[i].reshape(224, 224, 1)
        
        covid_sim = model.predict([[img_pneumonia], covid_template])
        normal_sim = model.predict([[img_pneumonia], normal_template])
        pneumonia_sim = model.predict([[img_pneumonia], pneumonia_template])
        
        sim = [covid_sim, normal_sim, pneumonia_sim]
        
        correct.append(2)
        predictions.append(np.argmin(sim))
    
    return np.asarray(correct), np.asarray(predictions)

def siamese_feature_extraction(model_siamese):
    """Create a new model based on the siamese network that takes an image as input and
    returns the calculated feature vector.
    """
    input_siamese = model_siamese.layers[2].get_input_at(0)
    output_siamese = model_siamese.layers[2].get_output_at(0)
    
    model = Model(inputs=input_siamese, outputs=output_siamese)
    return model

def model_final(model_siamese):
    """Final model used for testing."""
    
    input1 = Input(shape=(224, 224, 1))    # Input test image
    
    feature_extraction = siamese_feature_extraction(model_siamese)    # Feature extraction model
    
    feature_vec1 = feature_extraction(input1)
    feature_vec2 = Input(shape=256)    # Class template feature vector input
    
    l1 = Lambda(lambda tensors : keras.backend.abs(tensors[0] - tensors[1]), name="Lambda")([feature_vec1, feature_vec2])

    similarity = model_siamese.get_layer("Similarity")(l1)

    model_final = Model(inputs=[input1, feature_vec2], outputs=similarity)
    
    return model_final

covid_template = pickle.load(open(hp.path_pickle + "covid_template.pickle", "rb"))
normal_template = pickle.load(open(hp.path_pickle + "normal_template.pickle", "rb"))
pneumonia_template = pickle.load(open(hp.path_pickle + "pneumonia_template.pickle", "rb"))

model = model_final(model_siamese)

correct, prediction = test_siamese(model, X_covid_ts, X_normal_ts, X_pneumonia_ts, covid_template, normal_template, pneumonia_template)

bool_array = (correct == prediction)    # T/F array indication if the image was classified correctly
num_correct = np.count_nonzero(bool_array)
print(num_correct / len(correct))


In [ ]:
# Confusion matrix total numbers for siamese net
conf = confusion_matrix(correct, prediction)

seaborn.set(font_scale=1.1)
df_cm = pd.DataFrame(conf, index=["C", "N", "P"],  columns=["C", "N", "P"])
ax = seaborn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='d')
ax.set_xlabel("Predicted label", fontsize=14)
ax.set_ylabel("True label", fontsize=14)
plt.yticks(rotation=0)
#plt.savefig(hp.path_figs + "confSiameseNet_numbers.pdf")


In [ ]:
# Confusion matrix percentage for siamese model. Total accuracy 87%
conf = np.round(conf / np.sum(conf, axis=1).reshape(-1, 1), 2)

seaborn.set(font_scale=1.1)
df_cm = pd.DataFrame(conf, index=["C", "N", "P"],  columns=["C", "N", "P"])
ax = seaborn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
ax.set_xlabel("Predicted label", fontsize=14)
ax.set_ylabel("True label", fontsize=14)
plt.yticks(rotation=0)
#plt.savefig(hp.path_figs + "confSiameseNet_87p.pdf")

# 6.0 : Extra

## 6.1 : Image pre-processing and pickle

In [ ]:
# Read in images, resize to 224x224, convert to gray scale, store numpy arrays using pickle

path_train = "/content/drive/My Drive/Covid-19_datasets/data/train/"
path_test = "/content/drive/My Drive/Covid-19_datasets/data/test/"
path_pickle = "/content/drive/My Drive/Covid-19_datasets/data/Pickles/"
train_dict = pickle.load(open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/train_dict.pickle", "rb"))
test_dict = pickle.load(open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/test_dict.pickle", "rb"))

X_covid_tr = []
X_pneumonia_tr = []
X_normal_tr = []
X_covid_ts = []
X_pneumonia_ts = []
X_normal_ts = []

# Input dimensions used in COVID-Net
w = 224 
h = 224

for filename in os.listdir(path_train):
    if filename == ".ipynb_checkpoints":
        continue
    img = cv2.imread(path_train + filename, cv2.IMREAD_GRAYSCALE)   # Convert to grayscale??
    img = cv2.resize(img, (h, w), interpolation=cv2.INTER_AREA)
    # plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    if train_dict[filename] == "normal":
        X_normal_tr.append(img)
    elif train_dict[filename] == "pneumonia":
        X_pneumonia_tr.append(img)
    elif train_dict[filename] == "COVID-19":
        X_covid_tr.append(img)
    else:
        raise ValueError("Something wrong with dictionary")
    break   # Remove this to read in whole data set

print("Train finished")

for filename in os.listdir(path_test):
    if filename == ".ipynb_checkpoints":
        continue
    img = cv2.imread(path_test + filename, cv2.IMREAD_GRAYSCALE)   # Convert to grayscale?
    img = cv2.resize(img, (h, w), interpolation=cv2.INTER_AREA)
    # plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    if test_dict[filename] == "normal":
        X_normal_ts.append(img)
    elif test_dict[filename] == "pneumonia":
        X_pneumonia_ts.append(img)
    elif test_dict[filename] == "COVID-19":
        X_covid_ts.append(img)
    else:
        raise ValueError("Something wrong with dictionary")
    break   # Remove this to read in whole data set

X_covid_tr = np.asarray(X_covid_tr)
X_pneumonia_tr = np.asarray(X_pneumonia_tr)
X_normal_tr = np.asarray(X_normal_tr)
X_covid_ts = np.asarray(X_covid_ts)
X_pneumonia_ts = np.asarray(X_pneumonia_ts)
X_normal_ts = np.asarray(X_normal_ts)

#pickle.dump(X_covid_tr, open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/X_covid_tr.pickle", "wb"))
#pickle.dump(X_normal_tr, open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/X_normal_tr.pickle", "wb"))
#pickle.dump(X_pneumonia_tr, open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/X_pneumonia_tr.pickle", "wb"))
#pickle.dump(X_covid_ts, open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/X_covid_ts.pickle", "wb"))
#pickle.dump(X_normal_ts, open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/X_normal_ts.pickle", "wb"))
#pickle.dump(X_pneumonia_ts, open("/content/drive/My Drive/Covid-19_datasets/data/Pickles/X_pneumonia_ts.pickle", "wb"))

In [13]:
# Preprocess the unprocessed arrays stored in Pickles by normalizing the pixel
# values and then normalizing the data w.r.t. the training mean and std.
# The preprocessed data is stored in Pickles2

path_pickle = ""

# Training data
X_normal_tr = pickle.load(open(path_pickle + "X_normal_tr.pickle", "rb")) / 255.0
X_pneumonia_tr = pickle.load(open(path_pickle + "X_pneumonia_tr.pickle", "rb")) / 255.0
X_covid_tr = pickle.load(open(path_pickle + "X_covid_tr.pickle", "rb")) / 255.0
# Testing data
X_normal_val = pickle.load(open(path_pickle + "X_normal_val.pickle", "rb")) / 255.0
X_pneumonia_val = pickle.load(open(path_pickle + "X_pneumonia_val.pickle", "rb")) / 255.0
X_covid_val = pickle.load(open(path_pickle + "X_covid_val.pickle", "rb")) / 255.0
# Testing data
X_normal_ts = pickle.load(open(path_pickle + "X_normal_ts.pickle", "rb")) / 255.0
X_pneumonia_ts = pickle.load(open(path_pickle + "X_pneumonia_ts.pickle", "rb")) / 255.0
X_covid_ts = pickle.load(open(path_pickle + "X_covid_ts.pickle", "rb")) / 255.0

# Calculate mean and std of training data
X_tr = np.concatenate((X_normal_tr, X_pneumonia_tr, X_covid_tr), axis=0)
X_mean = np.mean(X_tr, axis=0)
X_std = np.std(X_tr, axis=0, ddof=1)

# Normalize all data w.r.t. training mean and std
X_normal_tr = (X_normal_tr - X_mean) / X_std
X_pneumonia_tr = (X_pneumonia_tr - X_mean) / X_std
X_covid_tr = (X_covid_tr - X_mean) / X_std

X_normal_val = (X_normal_val - X_mean) / X_std
X_pneumonia_val = (X_pneumonia_val - X_mean) / X_std
X_covid_val = (X_covid_val - X_mean) / X_std

X_normal_ts = (X_normal_ts - X_mean) / X_std
X_pneumonia_ts = (X_pneumonia_ts - X_mean) / X_std
X_covid_ts = (X_covid_ts - X_mean) / X_std

path_pickle = ""

pickle.dump(X_mean, open(path_pickle + "X_mean.pickle", "wb"))
pickle.dump(X_std, open(path_pickle + "X_std.pickle", "wb"))

#pickle.dump(X_normal_tr, open(path_pickle + "X_normal_tr.pickle", "wb"))
#pickle.dump(X_pneumonia_tr, open(path_pickle + "X_pneumonia_tr.pickle", "wb"))
#pickle.dump(X_covid_tr, open(path_pickle + "X_covid_tr.pickle", "wb"))

#pickle.dump(X_normal_val, open(path_pickle + "X_normal_val.pickle", "wb"))
#pickle.dump(X_pneumonia_val, open(path_pickle + "X_pneumonia_val.pickle", "wb"))
#pickle.dump(X_covid_val, open(path_pickle + "X_covid_val.pickle", "wb"))

#pickle.dump(X_normal_ts, open(path_pickle + "X_normal_ts.pickle", "wb"))
#pickle.dump(X_pneumonia_ts, open(path_pickle + "X_pneumonia_ts.pickle", "wb"))
#pickle.dump(X_covid_ts, open(path_pickle + "X_covid_ts.pickle", "wb"))

## 6.2 : Validation set creation and pickle

In [ ]:
# Create validation data

# Shuffle data
np.random.shuffle(X_covid_tr_full)
np.random.shuffle(X_covid_ts)
np.random.shuffle(X_normal_tr_full)
np.random.shuffle(X_normal_ts)
np.random.shuffle(X_pneumonia_tr_full)
np.random.shuffle(X_pneumonia_ts)

perc_val = 0.2
N_covid_val = int(perc_val * N_covid_tr_full)
N_normal_val = int(perc_val * N_normal_tr_full)
N_pneumonia_val = int(perc_val * N_pneumonia_tr_full)

# Covid images
mask_val = np.ones(N_covid_tr, dtype=bool)
mask_val[N_covid_val:] = False
mask_tr = [not mask for mask in mask_val]
X_covid_val = X_covid_tr_full[mask_val, :, :]
X_covid_tr = X_covid_tr_full[mask_tr, :, :]

# Normal images
mask_val = np.ones(N_normal_tr, dtype=bool)
mask_val[N_normal_val:] = False
mask_tr = [not mask for mask in mask_val]
X_normal_val = X_normal_tr_full[mask_val, :, :]
X_normal_tr = X_normal_tr_full[mask_tr, :, :]

# Pneumonia images
mask_val = np.ones(N_pneumonia_tr, dtype=bool)
mask_val[N_pneumonia_val:] = False
mask_tr = [not mask for mask in mask_val]
X_pneumonia_val = X_pneumonia_tr_full[mask_val, :, :]
X_pneumonia_tr = X_pneumonia_tr_full[mask_tr, :, :]


#path_pickle = "/content/drive/My Drive/Covid-19_datasets/data/Pickles/"
#pickle.dump(X_covid_tr, open(path_pickle + "X_covid_tr.pickle", "wb"))
#pickle.dump(X_covid_val, open(path_pickle + "X_covid_val.pickle", "wb"))
#pickle.dump(X_normal_tr, open(path_pickle + "X_normal_tr.pickle", "wb"))
#pickle.dump(X_normal_val, open(path_pickle + "X_normal_val.pickle", "wb"))
#pickle.dump(X_pneumonia_tr, open(path_pickle + "X_pneumonia_tr.pickle", "wb"))
#pickle.dump(X_pneumonia_val, open(path_pickle + "X_pneumonia_val.pickle", "wb"))


In [6]:
N_covid_val = len(X_covid_val)
N_normal_val = len(X_normal_val)
N_pneumonia_val = len(X_pneumonia_val)

index_normal = np.random.choice(N_normal_val, N_covid_val, replace=False)
index_pneumonia = np.random.choice(N_pneumonia_val, N_covid_val, replace=False)

X_covid_val = X_covid_val.reshape(-1, 224, 224, 1)
X_normal_val = X_normal_val[index_normal, :, :].reshape(-1, 224, 224, 1)
X_pneumonia_val = X_pneumonia_val[index_pneumonia, :, :].reshape(-1, 224, 224, 1)

Y_val = [0 for i in range(N_covid_val)]
Y_val.extend([1 for i in range(N_covid_val)])
Y_val.extend([2 for i in range(N_covid_val)])

X_val = np.concatenate((X_covid_val, X_normal_val, X_pneumonia_val))

p = np.random.permutation(len(Y_val))

Y_val = np.asarray(Y_val)

X_val = X_val[p]
Y_val = Y_val[p]

path_pickle = "Resized/"
pickle.dump(X_val, open(path_pickle + "X_val_single.pickle", "wb"))
pickle.dump(Y_val, open(path_pickle + "Y_val_single.pickle", "wb"))


In [16]:
# Load validation sets
X_covid_val_R = pickle.load(open("Resized/X_covid_val.pickle", "rb"))
X_pneumonia_val_R = pickle.load(open("Resized/X_pneumonia_val.pickle", "rb"))
X_normal_val_R = pickle.load(open("Resized/X_normal_val.pickle", "rb"))

X_covid_val = pickle.load(open("X_covid_val.pickle", "rb"))
X_pneumonia_val = pickle.load(open("X_pneumonia_val.pickle", "rb"))
X_normal_val = pickle.load(open("X_normal_val.pickle", "rb"))


In [31]:
# Load test sets
X_covid_ts_R = pickle.load(open("Resized/X_covid_ts.pickle", "rb"))
X_pneumonia_ts_R = pickle.load(open("Resized/X_pneumonia_ts.pickle", "rb"))
X_normal_ts_R = pickle.load(open("Resized/X_normal_ts.pickle", "rb"))

X_covid_ts = pickle.load(open("X_covid_ts.pickle", "rb"))
X_pneumonia_ts = pickle.load(open("X_pneumonia_ts.pickle", "rb"))
X_normal_ts = pickle.load(open("X_normal_ts.pickle", "rb"))

In [ ]:
# GradCAM, as implemented in https://github.com/eclique/keras-gradcam

import sys
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from keras import backend as K
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.python.framework import ops


H, W = 224, 224 # Input shape, defined by the model (model.input_shape)
# ---------------------------------------------------------------------

def load_image(path, x_mean, x_std):
    """Load and preprocess image."""
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (224, 224), interpolation=cv2.INTER_AREA)
    x = x.astype(dtype=np.float64)
    x /= 255
    x = (x - x_mean) / x_std
    
    return np.asarray(x)
   
def load_rgbimage(path, preprocess=True):
    """Load and preprocess image."""
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (224, 224), interpolation=cv2.INTER_AREA)
    
    return np.asarray(x)
   
def normalize(x):
    """L2 norm function"""
    return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)

def grad_cam(input_model, image, cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    #Target class output
    y_c = input_model.get_layer(layer_name).output
    
    #Map of activations
    conv_output = input_model.get_layer(layer_name).output
    #Get the gradient of the socre of the class (y_c) with respect of the map of activations.
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize 
    grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])
    #Mean of all the activation maps along the channels and the result obtained is the final class discriminative saliency map (weights).
    output, grads_val = gradient_function([image])  
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    weights = np.mean(grads_val, axis=(0, 1)) 
    cam = np.dot(output, weights)
    # Process CAM, ReLU applied
    cam = cv2.resize(cam, (W, H), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0) #ReLU
    cam_max = cam.max() 
    if cam_max != 0: 
        cam = cam / cam_max
    return cam


def compute_saliency(model, X_mean, X_std, sets, layer_name='block5_conv3', original=False, cls=-1, visualize=True):
    """Compute saliency using all three approaches.
        -layer_name: layer to compute gradients;
        -cls: class number to localize (-1 for most probable class).
    """
    preprocessed_input = sets[4]   #Input image is ovewritten here, can be changed by any image in the set.
    preim = preprocessed_input.reshape(-1,224,224,1)
    predictions = model.predict(x=preim)
    if cls == -1:
        cls = np.argmax(predictions)    
    gradcam = grad_cam(model, preim, cls, layer_name)
    
    if visualize:
        preprocessed_input = np.multiply(preprocessed_input, X_std) + X_mean
        preprocessed_input *= 255
        preprocessed_input = preprocessed_input.astype(dtype=np.int)
        
        plt.figure(figsize=(15, 10))
        plt.subplot(121)
        plt.title('GradCAM - Normal')
        plt.axis('off')
        plt.imshow(preprocessed_input, cmap="gray", vmin=0, vmax=255)
        plt.imshow(gradcam, cmap='jet', alpha=0.5)
        
        if original:
            plt.subplot(122)
            plt.title('Original')
            plt.axis('off')
            plt.imshow(preprocessed_input, cmap="gray", vmin=0, vmax=255)
        
        plt.savefig('Figures/normal_bad_4.jpg')
        plt.show()
        
    return gradcam



In [ ]:
# Load Mean and STD
X_mean = pickle.load(open("X_mean.pickle", "rb"))
X_std = pickle.load(open("X_std.pickle", "rb"))
#Compute GradCAM
gradcam1 = compute_saliency(model_single, layer_name='Output_before_SM', cls=-1, visualize=True, X_mean=X_mean, X_std=X_std, sets=X_normal_ts, original=True)